# A deeper dive into the Gauss-Levenberg-Marquardt (GLM) algorithm: weights and noise

The Gauss-Levenberg-Marquart (GLM) method is a gradient-based method used to search the objective function surface for its minimum value. It assumes that simulated values of observation targets vary continuously in response to changes in calibration-adjusted model parameters.  Two critical aspects of using ensemble-forms of GLM are how weights and observation noise are specified.  We will do this using the response surface of a simple 2-parameter form of the freyberg model

This notebook builds on the previous response-surface notebook - check it out for a discussion of the GLM lambda in the context of response surfaces...

### Admin
We have provided some pre-cooked PEST dataset files, wrapped around the modified Freyberg model. This is the same dataset introduced in the "freyberg_pest_setup" and "freyberg_k" notebooks. 

The functions in the next cell import required dependencies and prepare a folder for you. This folder contains the model files and a preliminary PEST setup. Run the cells, then inspect the new folder named "freyberg_mf6" which has been created in your tutorial directory. (Just press `shift+enter` to run the cells). 

In [ ]:
import sys
import os
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt;
import shutil

# sys.path.insert(0,os.path.join("..", "..", "dependencies"))
import pyemu
import flopy
assert "dependencies" in flopy.__file__
assert "dependencies" in pyemu.__file__
sys.path.insert(0,"..")
import herebedragons as hbd
import response_surface as resurf

plt.rcParams['font.size'] = 10
pyemu.plot_utils.font =10

Here we prep for the response surface runs:

In [ ]:
# folder containing original model files
org_d = os.path.join('..', '..', 'models', 'monthly_model_files_1lyr_newstress')
# a dir to hold a copy of the org model files
tmp_d = os.path.join('freyberg_mf6')

if os.path.exists(tmp_d):
    shutil.rmtree(tmp_d)
shutil.copytree(org_d,tmp_d)
# get executables
hbd.prep_bins(tmp_d)
# get dependency folders
hbd.prep_deps(tmp_d)
# run our convenience functions to prepare the PEST and model folder
hbd.prep_pest(tmp_d)

### Reminder - the modified-Freyberg model
Just a quick reminder of what the model looks like and what we are doing. 

It is a one-layer model. A river runs north-south, represented with the SFR package (green cells in the figure). On the southern border there is a GHB (cyan cells). No-flow cells are shown in black. Pumping wells are shown with red cells. 

Time-series of measured heads are available at the locations marked with black X's. River flux is also measured at three locations (headwater, tailwater and gage; not displayed).

The simulation starts with a steady state stress period, followed by twelve transient stress periods. These represent the historic period, for which measured data are available.

A subsequent twelve transient stress periods representing a period in the future. Modelling is undertaken to assess selected forecasts during the simulated period.

In [ ]:
hbd.plot_freyberg(tmp_d)

### The PEST Control File

You may  wish to explore the `freyberg_mf6` folder which has been created in the tutorial directory. In it you will find a PEST control file named `freyberg.pst`.

Let's use `pyemu` to load the PEST control file and check some details. 

In [ ]:
pst = pyemu.Pst(os.path.join(tmp_d, 'freyberg.pst'))
pst.par_names

In [ ]:
par = pst.parameter_data
par

We shall explore the effect of having two adjustable parameters: `hk1` and `rch0`. As we saw in the previous response surface notebook, with just groundwater level observations, these two parameters are strongly correlated in the objective function. If we use only head observations for calibration we are unable to achive a unique solution. 

(We shall change `hk1` parameter bounds just to make visualization easier.)

In [ ]:
par.loc['rch0', 'partrans'] = 'log'
par.loc['hk1', 'parlbnd'] = 1.5
par.loc['hk1', 'parubnd'] = 15

Re-write the control file:

In [ ]:
pst.write(os.path.join(tmp_d, 'freyberg.pst'),version=2)

The `response_surface.py` file in the tutorial folder contains a few functions to run `PESTPP-SWP` and make plots. These run `PESTPP-SWP` a few hundred times for a combination of `hk1` and `rch0` values.

In [ ]:
num_workers = 25

### Add Flux Observations

As we did in the previous response surface notebook, we know that if we add measurments of stream flow, we break the parameter correlation, so let's do that here too:

In [ ]:
pst = pyemu.Pst(os.path.join(tmp_d, 'freyberg.pst'))

In [ ]:
# set weights to gage-1 observations during calibration period
obs = pst.observation_data
obs.loc[:,"observed"] = False

obs_times = obs.loc[~obs['gage-1'].isnull(), 'gage-1'].astype(float).values
calib_times = [str(i) for i in obs_times if i<4018.5]
obs.loc[obs['gage-1'].isin(calib_times), 'weight'] = 0.003 # we know from our own experiments this is a good, error-based value that creates a balanced objective function...
obs.loc[obs.weight > 0,"observed"] = True

Re-write the control file.

In [ ]:
pst.write(os.path.join(tmp_d, 'freyberg.pst'),version=2)

Check that the response surface directories exist

In [ ]:
bal_resp_d = "bal_respsurf"
imbal_resp_d = "imbal_respsurf"
gw_resp_d = "gw_only_respsurf"
for d in [bal_resp_d,imbal_resp_d,gw_resp_d]:
    assert os.path.exists(d)," resp surf '"+d+"' missing you need to run the previous response surface notebook with the run flag set to 'True'"

And plot it up. As expected, we see the objective function surface funneling down to a single closed "region of attraction". We have achieved a unique solution.

In [ ]:
fig, ax, resp_surf = resurf.plot_response_surface(figsize=(7,7),WORKING_DIR=bal_resp_d,title="balanced groundwater levels and surface-water flow observations in $\\Phi$") #maxresp=1e3,

In [ ]:
fig, ax, resp_surf = resurf.plot_response_surface(figsize=(7,7),WORKING_DIR=imbal_resp_d,title="surface-water focused $\\Phi$") #maxresp=1e3,

In [ ]:
fig, ax, resp_surf = resurf.plot_response_surface(figsize=(7,7),WORKING_DIR=gw_resp_d,title="groundwater levels only in $\\Phi$") #maxresp=1e3,

Thats a pretty shocking difference, especially when we consider that this is a simple 2-D (ie 2-parameter) problem.  What's more, the goal of our data assimilation analyses is to navigate this surface to the (region around the) minimum.  Let's see how changing the weights changes the parameter posterior distribution

# Understanding how weights change iES results


This function just makes sure we are consistently using the same pest interface settings...

In [ ]:
def prep_pst(t_d="template",noptmax=6):
    if os.path.exists(t_d):
        shutil.rmtree(t_d)
    shutil.copytree(org_d,t_d)
    # get executables
    hbd.prep_bins(t_d)
    # get dependency folders
    hbd.prep_deps(t_d)
    # run our convenience functions to prepare the PEST and model folder
    hbd.prep_pest(t_d)
    pst = pyemu.Pst(os.path.join(t_d,"freyberg.pst"))
    obs = pst.observation_data
    obs_times = obs.loc[~obs['gage-1'].isnull(), 'gage-1'].astype(float).values
    calib_times = [str(i) for i in obs_times if i<4018.5]
    obs.loc[obs['gage-1'].isin(calib_times), 'weight'] = 0.003
    #obs.loc[pst.nnz_obs_names,"standard_deviation"] = 10. / obs.loc[pst.nnz_obs_names,"weight"]
    
    par = pst.parameter_data
    par.loc['rch0', 'partrans'] = 'log'
    par.loc['hk1', 'parlbnd'] = 1.5
    par.loc['hk1', 'parubnd'] = 15
    
    par.loc[pst.adj_par_names,"partrans"] = "none"
    par.loc[pst.adj_par_names,"parval1"] = (par.loc[pst.adj_par_names,"parlbnd"] + par.loc[pst.adj_par_names,"parubnd"]) / 2.0
    
    pst.control_data.noptmax = noptmax
    pst.write(os.path.join(t_d,"freyberg.pst"),version=2)

First let's run PESTPP-IES using the same weights as the balanced-weight groundwater level and surface-water flow response surface above:

In [ ]:
t_d = "template"
prep_pst(t_d=t_d)
pst = pyemu.Pst(os.path.join(t_d,"freyberg.pst"))
obs = pst.observation_data
pst.write(os.path.join(t_d,"freyberg.pst"),version=2)
m_d = "master_balweight"
pyemu.os_utils.start_workers(t_d,"pestpp-ies","freyberg.pst",num_workers=num_workers,worker_root=".",master_dir=m_d)

In [ ]:
resurf.plot_ies_and_resp_par_forecast_results(bal_resp_d,m_d,pst,title="iES with balanced $\\Phi$",fig_name="ies_resp_0.pdf")

GREAT SUCCESS!!!  Just what we wanted - each realization has a nice trajectory to the minimum of the misfit between simulated states and their observed counterparts.  Maybe you are wondering why each realization trajectory didn't converge to a single "best-fit" point on the response surface at the red star - this is a good question and we will discuss this soon!  So for now, just let that question burn in your mind... 

### optional exercise: using more iterations

In this simple example, where all of our modeling and analysis assumptions are satified (for the only time ever!), it's acceptable to run ies for many iterations to see that the parameter distributions "converge" and "stabilize".  Feel free to run the block below to prove it to yourself...

In [ ]:
t_d = "template_moreiters"
prep_pst(t_d=t_d,noptmax=20)
pst = pyemu.Pst(os.path.join(t_d,"freyberg.pst"))
pst.control_data.phiredstp = 0.0
obs = pst.observation_data
pst.write(os.path.join(t_d,"freyberg.pst"),version=2)
m_d = "master_balweight_moreiters"
pyemu.os_utils.start_workers(t_d,"pestpp-ies","freyberg.pst",num_workers=num_workers,worker_root=".",master_dir=m_d,port=4200)


In [ ]:
resurf.plot_ies_and_resp_par_forecast_results(bal_resp_d,m_d,pst,title="iES with more iterations")

Very similar to the previous analysis that used less iterations (and therefore fewer model runs).  BUT!   User beware:  using lots of iterations in real-world applied modeling is frought with trouble because so many of the assumptions that underpin the inverse/inference problem are violated (perfect model, perfect parameterization, perfect prior, perfect noise/liklihood, perfect inverse/inference algorithm, the list goes on and on...)

## Change the weights, change the results:

Now, in the typical groundwater modeling approach, lets mess with this success and use the modified weights we used for the surface-water-flow focused response surface above:

In [ ]:
prep_pst(t_d)
pst = pyemu.Pst(os.path.join(t_d,"freyberg.pst"))
obs = pst.observation_data
#obs.loc[pst.nnz_obs_names,"standard_deviation"] = 1. / obs.loc[pst.nnz_obs_names,"weight"]cobs = pst.observation_data
obs_times = obs.loc[~obs['gage-1'].isnull(), 'gage-1'].astype(float).values
calib_times = [str(i) for i in obs_times if i<4018.5]
ovals = obs.loc[obs['gage-1'].isin(calib_times), 'obsval']
obs.loc[obs['gage-1'].isin(calib_times), 'weight'] = 1./(ovals*0.01)
pst.write(os.path.join(t_d,"freyberg.pst"),version=2)
m_d = "master_largerweight"
pyemu.os_utils.start_workers(t_d,"pestpp-ies","freyberg.pst",num_workers=num_workers,worker_root=".",master_dir=m_d)

In [ ]:
resurf.plot_ies_and_resp_par_forecast_results(imbal_resp_d,m_d,pst,title="iES with Streamflow-Dominated $\\Phi$",fig_name="ies_resp_1.pdf")

Ouch - thats much uglier.  We can see that the trajectory across the response surface suffers from increased nonlinearity as the realizations navigate to a manufactured trough of dispair, and then just make a sharp change in direction.  #notgood.  Just for fun let's see if the PESTPP-IES multimodal solution process can help here - we will use basically all realizations in the ensemble and simply use the realization-local (ie multimodal) solution process to give us a better gradient approximation for each individual realization:

In [ ]:
pst.pestpp_options["ies_multimodal_alpha"] = 0.99        
pst.write(os.path.join(t_d,"freyberg.pst"),version=2)
m_d = "master_largerweight_mm"
pyemu.os_utils.start_workers(t_d,"pestpp-ies","freyberg.pst",num_workers=num_workers,worker_root=".",master_dir=m_d)

In [ ]:
resurf.plot_ies_and_resp_par_forecast_results(imbal_resp_d,m_d,pst,title="iES with Streamflow-Dominated $\\Phi$ and Realization-local Solver")

Ok, so the multimodal solution process can (as expected) deal with increasingly nonlinear inverse problems...

# Understanding how noise effects results

Results so far, the noise realizations used in the assimilation process were derived from the assumption that the noise standard deviation was the inverse of the assigned weights.  This is not an ideal situation and in real-world practice (where models are imperfect simulators), we should always seperate the weights and noise.  This can be done in many, many ways.  Herein, we will do this by supplying a `standard_deviation` column in the observation data.  For our first experiement, let's assume the noise standard deviation for groundwater levels is 5.0 and the noise standard deviation for sw flow is 33% of the flow value:

In [ ]:
t_d = "template"
prep_pst(t_d)
pst = pyemu.Pst(os.path.join(t_d,"freyberg.pst"))
obs = pst.observation_data
hnames = [n for n in pst.nnz_obs_names if "trgw" in n]
gnames = [n for n in pst.nnz_obs_names if "gage" in n]
assert len(hnames) + len(gnames) == pst.nnz_obs
obs.loc[hnames,"standard_deviation"] = 5.0
obs.loc[gnames,"standard_deviation"] = obs.loc[gnames,"obsval"] * 0.33
pst.write(os.path.join(t_d,"freyberg.pst"),version=2)
m_d = "master_bignoise"
pyemu.os_utils.start_workers(t_d,"pestpp-ies","freyberg.pst",num_workers=num_workers,worker_root=".",master_dir=m_d)

In [ ]:
resurf.plot_ies_and_resp_par_forecast_results(bal_resp_d,m_d,pst,title="iES with Balanced $\\Phi$\nand Greater Observation Noise",fig_name="ies_resp_2.pdf")

Yowza!  Thats a very different result! This spread in the posterior is a direct result of using noisy (aka "perturbed", aka "stochastic") observation values - remember each parameter realization is paired up with a set of these noise obs, so each parameter realization is aiming for different objective function minimum - more noise means bigger differences in realization objective function minima. This is how we explicitly include observation noise in the posterior results...

### No observation noise

Lets see what happens if we include no observation noise in this very simple over-determined inverse problem (foreshadowing: "over determined"!)

In [ ]:
t_d = "template_nonoise"
prep_pst(t_d=t_d,noptmax=10)
pst = pyemu.Pst(os.path.join(t_d,"freyberg.pst"))
pst.control_data.phiredstp = 0.0
obs = pst.observation_data
pst.pestpp_options["ies_no_noise"] = True
pst.write(os.path.join(t_d,"freyberg.pst"),version=2)
m_d = "master_orgweight_nonoise"
pyemu.os_utils.start_workers(t_d,"pestpp-ies","freyberg.pst",num_workers=num_workers,worker_root=".",master_dir=m_d,port=4200)

In [ ]:
resurf.plot_ies_and_resp_par_forecast_results(bal_resp_d,m_d,pst,title="iES with Balanced $\\Phi$\nand No Observation Noise",fig_name="ies_resp_3.pdf")

Remember early when we asked why each of the realizations doesnt converge to a single best-fit point on the response surface?  Well its the realizations of measurement that prevent this "collapse", at least in this synthetic case where we have only two parameters.  These results represent a #fail if we are trying to quantify posterior uncertainty, but we can learn something here:  In strongly over-determined inverse/inference problems (where there are very few parameters compared to the information content of the observation data), observation noise is a dominant source of posterior parameter uncertainty...but do we ever really have over-determined problems in the real-world unless we (modelers) force them to exist by under-parameterizing our inverse/inference problems...sounds dangerous!  

### Less noisy

Just to round-out our analyses, let's get a middle-ground noisy result for comparison:

In [ ]:
t_d = "template"
prep_pst(t_d)
pst = pyemu.Pst(os.path.join(t_d,"freyberg.pst"))
obs = pst.observation_data
obs = pst.observation_data
hnames = [n for n in pst.nnz_obs_names if "trgw" in n]
gnames = [n for n in pst.nnz_obs_names if "gage" in n]
assert len(hnames) + len(gnames) == pst.nnz_obs
obs.loc[hnames,"standard_deviation"] = 3.0
obs.loc[gnames,"standard_deviation"] = obs.loc[gnames,"obsval"] * 0.25
pst.write(os.path.join(t_d,"freyberg.pst"),version=2)
m_d = "master_mediumnoise"
pyemu.os_utils.start_workers(t_d,"pestpp-ies","freyberg.pst",num_workers=num_workers,worker_root=".",master_dir=m_d)


In [ ]:
resurf.plot_ies_and_resp_par_forecast_results(bal_resp_d,m_d,pst,title="iES with Balanced $\\Phi$\nand Less Observation Noise",fig_name="ies_resp_4.pdf")

Ok now it should be clear - the noise is controlling the posterior variance of both parameters - this is not unexpected in a well-posed inverse problem as both parameters are being strongly conditioned by all the observations.  But in all cases, thought must be put into noise, and, at the very least, weights and noise should be specified explicitly and separately, especially in situations where weights are being adjusted to balance contributions to the objective function.  

## Bonus material: understanding how ensemble methods cope with non-uniqueness

A major issue in applied groundwater modeling is the inability to uniquely identify all of the uncertain model inputs with the available data.  Up until this point in the notebook, our inverse problem has been "overdetermined", meaning that the information content in the observations we are trying match is (much) larger than the number of parameters we are using - this is rarely the case in practice - usually there are (many) parameters (or parameter combinations - see below!) that are not informed by the observation data.  Using subspace concepts, we would refer to this as a non-trivial null space and this source of uncertainty is a major contributor to posterior forecast uncertainty in applied groundwater modeling.  We saw this in pictures in the original response surface notebook when we graphed the response surface between `hk` and `recharge` when only groundwater level data were being assimilated (by removing the surface water flow observations, we can no longer uniquely estimate both hydraulic conductivity and recharge).  Lets see that again:

In [ ]:
fig, ax, resp_surf = resurf.plot_response_surface(figsize=(7,7),WORKING_DIR=gw_resp_d) #maxresp=1e3,

There is that trough of dispair where combinations of HK and recharge yield simulated groundwater levels that more-or-less reproduce observed groundwater levels.  Lets see how the ensemble smoother copes with this situation:

In [ ]:
t_d = "template"
prep_pst(t_d)
pst = pyemu.Pst(os.path.join(t_d,"freyberg.pst"))
obs = pst.observation_data
gnames = [n for n in pst.nnz_obs_names if "gage" in n]
obs.loc[gnames,"weight"] = 0.0
pst.write(os.path.join(t_d,"freyberg.pst"),version=2)
m_d = "master_nonunique"
pyemu.os_utils.start_workers(t_d,"pestpp-ies","freyberg.pst",num_workers=num_workers,worker_root=".",master_dir=m_d)

In [ ]:
resurf.plot_ies_and_resp_par_forecast_results(gw_resp_d,m_d,pst,title="iES with No Streamflow Obsevations in $\\Phi$",fig_name="ies_resp_noflux.pdf")

Is that what you expected to happen?  Its what we want to happen...The posterior parameter ensemble effectively represents/captures the non-uniqueness, as seen in the parameter histogram plots - the main thing we "learned" by assimilating the groundwater levels is that HK shouldn't be larger than around 10 m/d and that recharge shouldnt be much lower than about 0.9 m/d. Other than that, there are a wide range of HK-recharge combinations that yield a reasonble fit to the observed groundwater levels.  Still, even this modest parameter conditioning has reduced uncetainty for some forecasts as shown in the histogram plots. 

And just for completeness, let's see non-unique with no-noise:

In [ ]:
t_d = "template_nonu_nonoise"
prep_pst(t_d)
pst = pyemu.Pst(os.path.join(t_d,"freyberg.pst"))
obs = pst.observation_data
gnames = [n for n in pst.nnz_obs_names if "gage" in n]
obs.loc[gnames,"weight"] = 0.0
pst.pestpp_options["ies_no_noise"] = True
pst.write(os.path.join(t_d,"freyberg.pst"),version=2)
m_d = "master_nonunique_nonoise"
pyemu.os_utils.start_workers(t_d,"pestpp-ies","freyberg.pst",num_workers=num_workers,worker_root=".",master_dir=m_d)

In [ ]:
resurf.plot_ies_and_resp_par_forecast_results(gw_resp_d,m_d,pst,title="iES with No Streamflow Observations in $\\Phi$\nand No Observation Noise",fig_name="ies_resp_noflux_nonoise.pdf")

So even in nonunique settings, using explicit noise representation within ensemble method framework can be (very) important!

## Double bonus: experimenting with a realization-local upgrade solution for a highly non-unique inverse problem

one more time from the top: just for fun, let's run the the multimodal solver on this highly non-unique inverse problem.  Briefly, the multi-modal solver solves the Levenburg-Marquardt upgrade equation for each realization (one by one) and treats each realization as the center of it gradient approximation, rather than using a global mean gradient approximation that is shared across all realizations.  

Before running this experiment, try to visualize what you think the individual realization trajectories should look (remembering what the last experiment looked like, what lambda does to the solution trajectory, and how the multimodal solve forms a unique gradient estimate for each realization).  Got it?  Good...

In [ ]:
t_d = "template"
prep_pst(t_d)
pst = pyemu.Pst(os.path.join(t_d,"freyberg.pst"))
obs = pst.observation_data
gnames = [n for n in pst.nnz_obs_names if "gage" in n]
obs.loc[gnames,"weight"] = 0.0
pst.pestpp_options["ies_multimodal_alpha"] = 0.99
pst.write(os.path.join(t_d,"freyberg.pst"),version=2)
m_d = "master_nonunique_mm"
pyemu.os_utils.start_workers(t_d,"pestpp-ies","freyberg.pst",num_workers=num_workers,worker_root=".",master_dir=m_d)

In [ ]:
resurf.plot_ies_and_resp_par_forecast_results(gw_resp_d,m_d,pst)

Is that what you were expecting?  TBH this is not what I was expecting the first time I ran this experiment - it's a little counter-intuitive.  The key piece is to remember that a small lambda value is a more Newton-type solution that can cut across the gradient of the surface, and this, when combined with a more representative local gradient approximation for each realization, allows many realizations to "funnel" in toward the region of objective function minimum.  If you compare these trajectories to the previous experiment, you can see the value of the realization-local gradient approximation - in the previous experiment, all realizations in the lower right quadrant of parameter space essentially moved in the same trajectory (to the upper left), which is the optimal upgrade direction of the ensemble mean.  Whew! 